# PROYECTO FINAL: NullStorm

## 1. MODELO COMPUTER VISION

* Google Drive

In [28]:
from google.colab import drive
from google.colab import auth
#auth.authenticate_user()
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
!pip install easyocr


* Librerías necesarias

In [ ]:
pip install yolov5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.5/953.5 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 6.2 MB/s eta 0:00:00
  Cre

In [ ]:
from google.colab import files
from google.cloud import bigquery
import torch # Biblioteca para trabajar con modelos de aprendizaje profundo
import cv2 # Biblioteca para procesamiento de imágenes y video
import time # Para medir tiempos de ejecución
import re # Biblioteca para trabajar con expresiones regulares
import numpy as np # Biblioteca para cálculos numéricos
import easyocr # Biblioteca OCR para reconocimiento de texto
import os # Biblioteca para manejo de archivos y directorios
from IPython.display import Image, display # Para mostrar imágenes en Jupyter Notebook
import yolov5 # Biblioteca YOLOv5 para detección de objetos
import matplotlib.pyplot as plt
import re # Biblioteca Para trabajar con expresiones regulares
import random # Biblioteca Para generar datos aleatorios
from datetime import datetime, timedelta # Biblioteca Para trabajar con fechas y horas
import pandas as pd

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## 1.1. Función para cargar la imagen

In [ ]:
def img_load(img_path=None):
    """Función para cargar una imagen."""
    if img_path is None: # Si no se proporciona una ruta, solicita al usuario que cargue una imagen
        print("Por favor, selecciona una imagen del vehículo:")
        uploaded = files.upload() # Carga el archivo
        image_path = next(iter(uploaded)) # Obtiene la ruta del archivo cargado
        frame = cv2.imread(image_path) # Lee la imagen
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Convierte de BGR a RGB
        return frame
    else: # Si se proporciona una ruta, carga la imagen desde esa ruta
        frame = cv2.imread(img_path) # Lee la imagen
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Convierte de BGR a RGB
        return frame


## 1.2. Detección de objetos con YOLOv5

* Se cargan los modelos YOLOv5 para detección de vehículos y YOLOv5 para detección de matrículas.

In [ ]:
vehicle_model = torch.hub.load('ultralytics/yolov5', 'yolov5s') # Modelo YOLOv5 para detección de vehículos


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-3-2 Python-3.11.11 torch-2.5.1+cu124 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 100MB/s] 

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
#plate_model = torch.hub.load('keremberke/yolov5', 'yolov5m-license-plate') # Modelo YOLOv5 para detección de placas
plate_model = yolov5.load('keremberke/yolov5m-license-plate') # Modelo para detectar placas


config.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

best.pt:   0%|          | 0.00/42.1M [00:00<?, ?B/s]

* Función para la detección de objetos

In [ ]:
def detectx(frame, model):
    """Función para detectar objetos en una imagen usando un modelo YOLOv5."""
    results = model(frame) # Realiza la detección
    detections = results.xyxy[0]  # Obtiene las coordenadas de las detecciones

    if len(detections) == 0:
        return None # Si no hay detecciones, retorna None

    height, width, _ = frame.shape # Obtiene las dimensiones de la imagen
    center_x, center_y = width / 2, height / 2  # Calcula el centro de la imagen

    # Inicializar variables para la mejor detección
    closest_detection = None
    closest_distance = float('inf')

    # Iterar sobre las detecciones
    for detection in detections.tolist():
        x1, y1, x2, y2, conf, cls = detection # Extrae información de la detección
        # Calcular el centro de la caja delimitadora
        box_center_x = (x1 + x2) / 2
        box_center_y = (y1 + y2) / 2
        # Calcular la distancia al centro de la imagen
        distance = ((box_center_x - center_x) ** 2 + (box_center_y - center_y) ** 2) ** 0.5
        # Guardar la detección más cercana al centro
        if distance < closest_distance:
            closest_distance = distance
            closest_detection = detection

    return closest_detection if closest_detection is not None else None

* Función para devolver las coordenadas del objeto drawDetectedDiamonds

In [ ]:
def coords(closest_detection):
    """Función para obtener las coordenadas de la detección más cercana."""
    if closest_detection is None:
        return None
    x1, y1, x2, y2, conf, cls = closest_detection # Extrae información de la detección

    # Filtrar por confianza
    if conf >= 0.55: # Si la confianza es mayor o igual a 0.55
      coords = [int(x1), int(y1), int(x2), int(y2)] # Retorna las coordenadas como enteros
      return coords

    return None

## 1.3. Funciones para limpiar el texto

* Incluimos formatos de placas de varios países

In [ ]:
formatos_placas = [
    (r'^[A-Z]{2}\d{3,4}[A-Z]{2}$', 'Francia'),
    (r'^[A-Z]{1,2}[A-Z]{1,2}\d{3,4}$', 'Alemania'),
    (r'^[A-Z]{2}\d{3}[A-Z]{2}$', 'Italia'),
    (r'^[A-Z]{2}\d{2}[A-Z]{3}$', 'Reino Unido'),
    (r'^[A-Z]{3}\d[A-Z]{2}\d{4}$', 'Formato Especial')
]

def identificar_formato_placa(placa):
    for patron, pais in formatos_placas:
        if re.match(patron, placa):
            return pais
    return None

* Se crea una función para filtrar el texto reconocido comprobando que sólo tiene letras y números.

In [ ]:
def filter_plate_text(plate_text):
    """Función para filtrar el texto de la placa, dejando solo letras y números."""
    if isinstance(plate_text, list):
        plate_text = ''.join(plate_text) # Si es una lista, la convierte a string
    filtered_text = re.sub(r'[^A-Za-z0-9]', '', plate_text) # Elimina caracteres que no sean letras o números
    return filtered_text.upper() # Convierte el texto a mayúsculas

* Se crea otra función para filtrar el texto basado en el tamaño de la región.

In [ ]:
def filter_text(region, ocr_result, region_threshold):
    """Función para filtrar el texto reconocido por OCR basado en el tamaño de la región."""
    rectangle_size = region.shape[0]*region.shape[1] # Calcula el tamaño de la región
    plate = []
    for result in ocr_result:
        length = np.sum(np.subtract(result[0][1], result[0][0])) # Calcula la longitud del texto
        height = np.sum(np.subtract(result[0][2], result[0][1])) # Calcula la altura del texto
        if length*height / rectangle_size > region_threshold: # Si el área del texto es mayor que el umbral
            plate.append(result[1]) # Añade el texto a la lista
    return plate


* Se crea otra función para validar el formato de la placa.

In [ ]:
def validar_placa(placa):
    if placa is None:
        return False

    # Primero, verificar formatos internacionales
    for patron, _ in formatos_placas:
        if re.match(patron, placa):
            return True

    # Si no coincide con formatos internacionales, verificar el formato español
    return re.match(r'^\d{4}[BCDFGHJKLMNPRTSVWXYZ]{3}$', placa) is not None

* Se crea otra función para corregir errores comunes en la placa.

In [31]:
def corregir_placa(placa):
    # Si la placa es None o una cadena vacía, retornar una cadena vacía
    if not placa:
        return ""

    # Si la placa es una lista, convertirla a string uniendo sus elementos sin espacios
    if isinstance(placa, list):
        placa = ''.join(placa)

    # Primero, verificar si coincide con algún formato internacional
    for patron, _ in formatos_placas:
        if re.match(patron, placa):
            return placa  # Si coincide con un formato internacional, la devolvemos tal cual

    # Si no coincide con formatos internacionales, intentamos corregir al formato español

    # Reemplazar 'L' por '4' si está en los primeros 4 caracteres (parte numérica)
    placa = ''.join(['4' if c == 'L' and i < 4 else c for i, c in enumerate(placa)])

    # Extraer todos los dígitos de la placa
    numeros = ''.join(c for c in placa if c.isdigit())
    # Extraer todas las letras de la placa
    letras = ''.join(c for c in placa if c.isalpha())

    # Si hay 5 o más números, tomar los últimos 4
    # Si hay menos de 4, rellenar con ceros a la izquierda
    numeros = numeros[-4:].zfill(4)

    # Tomar las primeras 3 letras, si hay menos, rellenar con 'A' a la derecha
    letras = letras[:3].ljust(3, 'A')

    # Retornar la placa corregida en el formato "NNNNLLL"
    return f"{numeros}{letras}"

## 1.4. Reconocimiento de texto con EasyOCR

In [ ]:
def recognize_plate_easyocr(img, coords, reader, region_threshold):
    """Función para reconocer el texto de la placa usando EasyOCR."""
    xmin, ymin, xmax, ymax = coords
    nplate = img[int(ymin):int(ymax), int(xmin):int(xmax)] # Recorta la región de la placa
    ocr_result = reader.readtext(nplate) # Realiza OCR en la región
    text = filter_text(region=nplate, ocr_result=ocr_result, region_threshold=region_threshold) # Filtra el texto
    if len(text) == 1:
        text = text[0].upper() # Si solo hay un texto, lo convierte a mayúsculas

    # Aplicar filter_plate_text antes de corregir_placa
    text = filter_plate_text(text)

    # Aplicar corrección y validación
    text_corregido = corregir_placa(text)
    if validar_placa(text_corregido):
        return text_corregido
    return text

## 1.5. Extracción de la información del vehículo

In [ ]:
def vehicle_info(frame, vehicle_detection, plate_detection):
    """Función para extraer información del vehículo y la placa."""
    vehicle_classes = ['car', 'motorcycle']  # Clases de vehículos que queremos detectar
    extracted_info = {}  # Diccionario para almacenar información extraída
    translation_dict = {'car': 'carro', 'motorcycle': 'motocicleta'}  # Diccionario para traducir al español
    type_dict = {'car': '1', 'motorcycle': '2'}  # Diccionario para asignar un tipo numérico a cada clase

    def translate_vehicle_class(cls_name):
        """Función interna para traducir el nombre de la clase del vehículo."""
        return translation_dict.get(cls_name, cls_name)

    if vehicle_detection:  # Verifica si se detectó un vehículo
        x1, y1, x2, y2, conf, cls = vehicle_detection  # Extrae información de la detección
        if conf >= 0.55 and vehicle_model.names[int(cls)] in vehicle_classes:  # Si la confianza es alta y la clase es válida
            vehicle_name = translate_vehicle_class(vehicle_model.names[int(cls)])  # Traduce el nombre del vehículo
            vehicle_type = type_dict[vehicle_model.names[int(cls)]]  # Obtiene el tipo numérico del vehículo
            extracted_info["vehicle_name"] = vehicle_name  # Almacena el nombre del vehículo en el diccionario
            extracted_info["vehicle_type"] = vehicle_type  # Almacena el tipo numérico en el diccionario

    if plate_detection:  # Verifica si se detectó una placa
        x1, y1, x2, y2, conf, cls = plate_detection  # Extrae información de la detección
        if conf >= 0.55:  # Si la confianza es alta
            coords = [int(x1), int(y1), int(x2), int(y2)]  # Obtiene las coordenadas de la placa
            plate_num = recognize_plate_easyocr(img=frame, coords=coords, reader=EASY_OCR, region_threshold=OCR_TH)  # Realiza OCR en la región de la placa
            extracted_info["plate"] = plate_num  # Almacena el número de placa en el diccionario

    return extracted_info

## 1.6. Generar fecha ficticia para entrada y salida del vehículo

In [ ]:
def generar_fecha_hora():
    """Función para generar una fecha y hora actual."""
    return datetime.now()


## 1.7. Generar datos aleatorios para cada matrícula

In [ ]:
def generar_datos_random(matricula, vehicle_type=None, vehicle_name=None):
    """
    Función para generar datos aleatorios para un vehículo.

    Args:
    matricula (str): Número de matrícula detectado.
    vehicle_type (str): Tipo de vehículo detectado ('1' para coche, '2' para motocicleta).

    Returns:
    dict: Información generada aleatoriamente para el vehículo.
    """
    marcas_modelos = {  # Diccionario de marcas y sus modelos correspondientes
        "Toyota": ["Supra", "Celica", "Yaris"],
        "Ford": ["GT", "Focus", "Mustang"],
        "Honda": ["NSX", "Civic", "CR-V"],
        "Nissan": ["GTR", "350z", "240sx"],
        "Audi": ["RS7", "R8", "S5"],
        "BMW": ["330e", "M4", "i8"],
        "Mercedes": ["C63S", "GT", "A45"],
        "Tesla": ["3", "Y", "X"],
        "Porsche": ["911", "Cayman", "Taycan"],
    }
    colores = ["Rojo", "Azul", "Negro", "Blanco", "Gris", "Amarillo", "Verde"]  # Lista de colores posibles
    tarifas = ["tarifa_001", "tarifa_002", "tarifa_005"]  # Lista de tarifas posibles
    parking_camaras = {  # Diccionario de parkings y sus cámaras correspondientes
        "parking_001": ("camara_in_001", "camara_out_001"),
        "parking_002": ("camara_in_002", "camara_out_002"),
        "parking_005": ("camara_in_005", "camara_out_005"),
    }

    fecha_entrada = generar_fecha_hora()  # Genera una fecha y hora de entrada
    parking_id = random.choice(list(parking_camaras.keys()))  # Elige un parking aleatorio
    camara_in_id, camara_out_id = parking_camaras[parking_id]  # Obtiene las cámaras del parking
    camara_seleccionada = {"camara_in_id": "", "camara_out_id": ""}
    if random.choice([True, False]):  # Elige aleatoriamente entre cámara de entrada o salida
        camara_seleccionada["camara_in_id"] = camara_in_id
    else:
        camara_seleccionada["camara_out_id"] = camara_out_id

    marca = random.choice(list(marcas_modelos.keys()))  # Elige una marca aleatoria
    modelo = random.choice(marcas_modelos[marca])  # Elige un modelo aleatorio de la marca
    color = random.choice(colores)  # Elige un color aleatorio
    tarifa_id = random.choice(tarifas)  # Elige una tarifa aleatoria

    # Manejo del tipo de vehículo y su nombre
    if vehicle_type is None or vehicle_type not in ["1", "2"]:
        print("Advertencia: No se detectó el tipo de vehículo o es inválido. Se asume tipo 1 (carro).")
        tipo_vehiculo_id = "1"
        tipo_vehiculo_nombre = "carro"
    else:
        tipo_vehiculo_id = vehicle_type
        tipo_vehiculo_nombre = vehicle_name

    car_info = {
        "matricula_id": matricula,
        "parking_id": parking_id,
        "fecha_registro_dt": fecha_entrada.strftime("%Y-%m-%d %H:%M:%S"),
        "tarifa_id": tarifa_id,
        "marca_ds": marca,
        "modelo_ds": modelo,
        "color_ds": color,
        "tipo_vehiculo_id": tipo_vehiculo_id,
        "tipo_vehiculo_nombre": tipo_vehiculo_nombre,
        **camara_seleccionada
    }
    return car_info

## 1.8. Insertar los datos de la matrícula en la base de datos

In [ ]:
def insertar_matricula(car_info):
    """Función para insertar datos en BigQuery."""
    client = bigquery.Client() # Crea un cliente de BigQuery
    table_id = "aiparking-451016.00_aiparking.accesos" # ID de la tabla en BigQuery
    fila = {
        "matricula_id": car_info["matricula_id"],
        "parking_id": car_info["parking_id"],
        "fecha_registro_dt": car_info["fecha_registro_dt"],
        "camara_in_id": car_info["camara_in_id"],
        "camara_out_id": car_info["camara_out_id"],
        "tarifa_id": car_info["tarifa_id"],
        "marca_ds": car_info["marca_ds"],
        "modelo_ds": car_info["modelo_ds"],
        "color_ds": car_info["color_ds"],
        "tipo_vehiculo_id": car_info["tipo_vehiculo_id"],
        "tipo_vehiculo_nombre": car_info["tipo_vehiculo_nombre"]
    }

    # Insertar la fila
    #errores = client.insert_rows_json(table_id, [fila])

    #if errores == []:
        #print("Matrícula insertada correctamente.")
    #else:
        #print("Errores al insertar:", errores)

## 1.9. PIPELINE COMPUTER VISION MODEL

In [ ]:
EASY_OCR = easyocr.Reader(['en']) # Inicializar EasyOCR para idioma inglés
OCR_TH = 0.2 # Umbral de confianza para el OCR

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
def main_CV(img_path=None):
    """Función principal que orquesta todo el proceso de detección y procesamiento."""
    frame = img_load(img_path)  # Carga imagen desde ruta proporcionada o cargada por usuario

    vehicle_detection = detectx(frame, model=vehicle_model)  # Detecta vehículos usando YOLOv5
    print("Detección de vehículo:", vehicle_detection)

    plate_detection = detectx(frame, model=plate_model)  # Detecta placas usando YOLOv5
    print("Detección de placa:", plate_detection)

    plate_info = vehicle_info(frame, vehicle_detection, plate_detection)  # Extrae información sobre vehículos y placas

    if 'plate' in plate_info:  # Verifica si hay información sobre placas
        vehicle_type = plate_info.get('vehicle_type')  # Obtiene el tipo de vehículo si existe
        vehicle_name = plate_info.get('vehicle_name')  # Obtiene el nombre del vehículo si existe

        if vehicle_type:
            print(f"Vehículo detectado: {vehicle_name}, tipo de vehículo {vehicle_type}")
        else:
            print("No se detectó el tipo de vehículo. Se usarán valores por defecto.")

        info = generar_datos_random(plate_info['plate'], vehicle_type, vehicle_name)  # Genera datos aleatorios usando la matrícula y el tipo de vehículo

        insertar_matricula(info)  # Inserción en BigQuery

        return info  # Devuelve la información generada
    else:
        print("No se detectó placa.")
        return None  # Retorna None si no se detectó placa

In [53]:
info2 = main_CV() # Ejecuta la función principal
print(info2) # Imprime la información resultante

Por favor, selecciona una imagen del vehículo:


Saving E1,CAM1,250210005134979,8485MSR,9992.jpg to E1,CAM1,250210005134979,8485MSR,9992.jpg


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Detección de vehículo: None


/usr/local/lib/python3.11/dist-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Detección de placa: [503.69097900390625, 230.44918823242188, 690.7267456054688, 284.3781433105469, 0.8453854918479919, 0.0]
No se detectó el tipo de vehículo. Se usarán valores por defecto.
Advertencia: No se detectó el tipo de vehículo o es inválido. Se asume tipo 1 (carro).
{'matricula_id': '8485MSR', 'parking_id': 'parking_001', 'fecha_registro_dt': '2025-03-02 21:01:58', 'tarifa_id': 'tarifa_002', 'marca_ds': 'Mercedes', 'modelo_ds': 'A45', 'color_ds': 'Rojo', 'tipo_vehiculo_id': '1', 'tipo_vehiculo_nombre': 'carro', 'camara_in_id': '', 'camara_out_id': 'camara_out_001'}


## Creamos un archivo donde almacenamos el  nombre de las imágenes y las placas respectivas

In [48]:
# Ruta a la carpeta de imágenes
path = '/content/drive/My Drive/PROYECTO/TOTAL/IMAGENESTOTAL'

# Lista para almacenar los datos
data = []

# Recorrer los archivos en la carpeta
for filename in os.listdir(path):
    if filename.endswith('.jpg'):
        # Dividir el nombre del archivo
        parts = filename.split(',')
        if len(parts) >= 4:
            placa = parts[3]
            data.append({'Nombre del archivo': filename, 'Placa': placa})

# Crear DataFrame
df = pd.DataFrame(data)

# Guardar como CSV
df.to_csv('/content/drive/MyDrive/PROYECTO/TOTAL/resultado.csv', index=False)

## Adicionamos la columna placa detectada al archivo creado.

In [49]:
from tqdm import tqdm

# Cargar el CSV existente
csv_path = '/content/drive/MyDrive/PROYECTO/TOTAL/resultado.csv'
df = pd.read_csv(csv_path)

# Directorio que contiene las imágenes
image_directory = '/content/drive/My Drive/PROYECTO/TOTAL/IMAGENESTOTAL'

In [50]:
def process_images(df, image_directory):
    # Lista para almacenar las placas detectadas
    detected_plates = []

    # Iterar sobre cada fila del DataFrame, usando tqdm para mostrar una barra de progreso
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Procesando imágenes"):
        img_path = os.path.join(image_directory, row['Nombre del archivo']) # Construye la ruta completa de la imagen
        if os.path.exists(img_path): # Verifica si la imagen existe en la ruta especificada
            info = main_CV(img_path) # Procesa la imagen usando la función main_CV
            if info and 'matricula_id' in info: # Si se obtuvo información y contiene 'matricula_id', añade la placa detectada
                detected_plates.append(info['matricula_id'])
            else:
                detected_plates.append(None) # Si no se detectó placa, añade None
        else:
            print(f"No se encontró la imagen: {img_path}")
            detected_plates.append(None) # Si la imagen no existe, imprime un mensaje y añade None

    df['Placa detectada'] = detected_plates # Añade una nueva columna 'Placa detectada' al DataFrame con las placas detectadas
    return df

# Procesar las imágenes y actualizar el DataFrame
df_updated = process_images(df, image_directory)

# Guardar el DataFrame actualizado
df_updated.to_csv(csv_path, index=False)
print("CSV actualizado con las placas detectadas.")

Output hidden; open in https://colab.research.google.com to view.

## Calculamos métricas

In [51]:
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score
from difflib import SequenceMatcher

csv_path = '/content/drive/MyDrive/PROYECTO/TOTAL/resultado.csv' # Ruta del archivo CSV con los resultados
df = pd.read_csv(csv_path) # Cargar el archivo CSV en un DataFrame de pandas

# Función para analizar errores entre la placa real y la detectada
def analizar_errores(placa_real, placa_detectada):
    errores = []
    if pd.isna(placa_real) or pd.isna(placa_detectada): # Si alguna de las placas es NaN, se considera como "No detección"
        return ['No detección']
    # Convertir ambas placas a strings
    placa_real = str(placa_real)
    placa_detectada = str(placa_detectada)
    # Comparar cada carácter de las placas
    for char_real, char_detectado in zip(placa_real, placa_detectada.ljust(len(placa_real))):
        if char_real != char_detectado:
            errores.append(f"{char_real}->{char_detectado}")
    # Si la placa detectada es más larga, añadir los caracteres extra como error
    if len(placa_detectada) > len(placa_real):
        errores.extend([f"->{''.join(placa_detectada[len(placa_real):])})"])
    return errores

# Aplicar la función de análisis de errores a cada fila del DataFrame
df['errores'] = df.apply(lambda row: analizar_errores(row['Placa'], row['Placa detectada']), axis=1)
errores_comunes = pd.Series([error for errores in df['errores'] for error in errores]).value_counts() # Contar los errores más comunes

print("Errores más comunes:")
print(errores_comunes.head())

# Métricas adicionales
y_true = df['Placa'].fillna('No_placa')
y_pred = df['Placa detectada'].fillna('No_deteccion')

# Calcular métricas de precisión, recall y F1-score
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

print(f"\nMétricas adicionales:")
print(f"Precisión: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Función para calcular la similaridad entre placas reales y detectadas
# Aplica la fórmula: ratio = 2.0 * M / T
# M es el número de coincidencias entre las secuencias
# T es el número total de elementos en ambas secuencias.
def calcular_similaridad(placa_real, placa_detectada):
    if pd.isna(placa_real) or pd.isna(placa_detectada):
        return 0
    return SequenceMatcher(None, str(placa_real), str(placa_detectada)).ratio()

# Aplicar la función de similaridad a cada fila del DataFrame
df['similaridad'] = df.apply(lambda row: calcular_similaridad(row['Placa'], row['Placa detectada']), axis=1)

print("\nEstadísticas de similaridad:")
print(df['similaridad'].describe())

# Guardar los resultados del análisis en un nuevo archivo CSV
df.to_csv('/content/drive/MyDrive/PROYECTO/TOTAL/resultados_analisis.csv', index=False)
print("\nSe han guardado los resultados del análisis en 'resultados_analisis.csv'")

Errores más comunes:
W->I    20
4->0    20
W->H    18
G->6    17
T->I    16
Name: count, dtype: int64

Métricas adicionales:
Precisión: 0.87
Recall: 0.79
F1-Score: 0.81

Estadísticas de similaridad:
count    2242.000000
mean        0.947713
std         0.139770
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: similaridad, dtype: float64

Se han guardado los resultados del análisis en 'resultados_analisis.csv'


## 1. MODELO LLM

In [ ]:
from openai import OpenAI
import json
import getpass

api_key = getpass.getpass("Enter your OpenAI API Key:")

client_openai = OpenAI(api_key = api_key)

Enter your OpenAI API Key:··········


### 1.1. Input del usuario

In [ ]:
def user_input():
    question = input("Escriba su consulta:")
    return question

### 1.2. Prompt Engineering con ChatGPT

In [ ]:
def input_format(question, model="gpt-3.5-turbo", temperature=0):

    prompt_template = """
Eres un experto de SQL para una base de datos de registro de vehiculos. Tu tarea es convertir una pregunta de lenguaje natural a una consulta SQL valida.

## **ESTRUCTURA DE LA BASE DE DATOS**
Nombre del proyecto: 'aiparking-451016'
Nombre de la tabla: '00_aiparking.accesos'

- matricula_id STRING
- parking_id STRING
- fecha_registro_dt DATETIME
- camara_in_id STRING
- camara_out_id STRING
- tipo_vehiculo_id STRING
- marca_ds STRING
- modelo_ds STRING
- color_ds STRING
- tarifa_id STRING

## **TAREA**
Responde UNICAMENTE con la consulta en SQL.

## **EJEMPLOS**
Pregunta: "Cual es el modelo del vehiculo con la matricula 5489 HEB?"
Respuesta: SELECT modelo_ds FROM `aiparking-451016.00_aiparking.accesos` WHERE matricula_id = '5489 HEB';

Pregunta: "Coches de color Gris, Blanco o Azul y que sean de tipo 1"
Respuesta: SELECT * FROM `aiparking-451016.00_aiparking.accesos` WHERE color_ds IN ('Gris', 'Blanco', 'Azul') AND tipo_vehiculo_id = '1';

Pregunta: "A qué hora entró el coche con matrícula 1234ABC el día de ayer?"
Respuesta: SELECT fecha_registro_dt FROM `aiparking-451016.00_aiparking.accesos` WHERE matricula_id = '1234ABC' AND DATE(fecha_registro_dt) = DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY);;

## **NUEVA PREGUNTA DEL USUARIO:**
"{question}"

## **RESPUESTA EN SQL:**
"""
    prompt = prompt_template.format(question=question)
    messages = [{"role": "user", "content": prompt}]
    response = client_openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

    sql_query = response.choices[0].message.content
    return sql_query


* Ejemplo 1

In [ ]:
question_1 = "qué coches con matrícula que acaba en L entraron al parking el día 25/01/2025 entre las 07:00am y las 09:30am?"

In [ ]:
response_1 = input_format(question_1)
print(response_1)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: 1.\tsk-pr***********************************************************************************************************************************************************OaEA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

### 1.3. Acceso a la Database de BigQuery

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import bigquery

# Creamos un cliente de BigQuery
client = bigquery.Client(project='aiparking-451016')

In [ ]:
import pandas as pd

def sql_ans(query):

      # Consulta SQL
      query_job = client.query(query)

      results = query_job.result()

      # Convertir los resultados a un DataFrame de Pandas
      df = results.to_dataframe()

      return df

### 1.4. Pipeline

In [ ]:
def main_LLM(question=None):

    # 1. Pregunta del usuario
    if question is None:
      question = user_input()

    # 2. Conversión lenguaje natural a SQL
    query = input_format(question=question, model="gpt-3.5-turbo", temperature=0)

    # 3. Acceso al Database de BigQuery
    answer = sql_ans(query)

    # 6. Mostrar la salida
    print(answer)

    return answer

In [ ]:
main_LLM()

Escriba su consulta:que marca y modelo es el coche 3483JLFI?
   marca_ds modelo_ds
0  Mercedes      C63S
1     Tesla         X


,marca_ds,modelo_ds
0,Mercedes,C63S
1,Tesla,X
